# Hyperparameter Tuning for ANN-4

In [ ]:
# importing necessary public modules
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Input, Model
import tensorflow as tf
!pip install keras-tuner
import keras_tuner as kt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 8.1 MB/s 


In [ ]:
# check to see if GPU is in use
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# connection to google drive + import data
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/My Drive/content")

# importing Heston data
data = np.genfromtxt('heston_data.csv', delimiter=',')

# inputs: converting spot price + strike price to moneyness (s/k)
inputs = data[:,:9] 
X = np.array([np.array([x[0]/x[1], x[2],x[3],x[4], x[5], x[6], x[7], x[8]])for x in inputs])

# outputs: scaling the prices byt the strike price (p/k)
y_calls = np.array(data[:,-2]) / data[:,1]
y_puts = np.array(data[:,-1]) / data[:,1]

# training data
X_train = X[:150000]
y_calls_train = y_calls[:150000]
y_puts_train = y_puts[:150000]

# validation data
X_val = X[150000:175000]
y_calls_val = y_calls[150000:175000]
y_puts_val = y_puts[150000:175000]

# testing data
X_test = X[175000:]
y_calls_test = y_calls[175000:]
y_puts_test = y_puts[175000:]

# shape check
print(X_train.shape)
print(y_calls_train.shape)
print(y_puts_train.shape)
print(X_val.shape)
print(y_calls_val.shape)
print(y_puts_val.shape)
print(X_test.shape)
print(y_calls_test.shape)
print(y_puts_test.shape)

Mounted at /content/drive
(150000, 8)
(150000,)
(150000,)
(25000, 8)
(25000,)
(25000,)
(25000, 8)
(25000,)
(25000,)


In [ ]:
# Random Search method
def build_model(hp):
  inputs = Input(shape=(8,), name='input')

  activations = hp.Choice('activation',['relu', 'tanh', 'selu'])
  dropout = hp.Choice('dropout', [True, False])

  x = inputs
  for i in range(hp.Int('num_layers', 1, 10, step=1)):
    x = Dense(units=hp.Int('units',
                           min_value = 50,
                           max_value = 800,
                           step = 50),
              kernel_initializer='glorot_uniform',
              activation = activations)(x)
    if dropout == True:
      x = Dropout(0.2)(x)
    else:
      x = x
    
                      
  call = Dense(1, name='call_output')(x)
  put = Dense(1, name='put_output')(x)

  opt = keras.optimizers.Adam(hp.Choice('learning_rate',
                                        [1e-2, 1e-3, 1e-4, 1e-5]))

  model = Model(inputs=inputs, outputs=[call, put])
  model.compile(loss='mse', optimizer=opt)

  return model

tuner = kt.RandomSearch(build_model,
                     objective='val_loss',
                     max_trials=40,
                     #directory='monte_carlo_heston_dir',
                     project_name = 'heston_dir')

INFO:tensorflow:Reloading Oracle from existing project ./heston_dir/oracle.json


In [ ]:
# summary of hyperparameter spaces for Random Search to consider
tuner.search_space_summary()

Search space summary
Default search space size: 5
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'selu'], 'ordered': False}
dropout (Choice)
{'default': 1, 'conditions': [], 'values': [1, 0], 'ordered': True}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 10, 'step': 1, 'sampling': None}
units (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 800, 'step': 50, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001, 1e-05], 'ordered': True}


In [ ]:
# searching algorithm 
tuner.search(X_test, [y_calls_test, y_puts_test],
             epochs = 40,
             validation_data=(X_val, [y_calls_val, y_puts_val]))

Trial 40 Complete [00h 03m 34s]
val_loss: 8.63887180457823e-05

Best val_loss So Far: 2.0125140508753248e-05
Total elapsed time: 02h 16m 48s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# summary of random search to show best settings
tuner.results_summary()

Results summary
Results in ./heston_dir
Showing 10 best trials
Trial summary
Hyperparameters:
activation: relu
dropout: 0
num_layers: 9
units: 500
learning_rate: 1e-05
Score: 2.0125140508753248e-05
Trial summary
Hyperparameters:
activation: tanh
dropout: 0
num_layers: 2
units: 250
learning_rate: 0.001
Score: 3.477330756140873e-05
Trial summary
Hyperparameters:
activation: selu
dropout: 0
num_layers: 7
units: 350
learning_rate: 0.0001
Score: 3.7531848647631705e-05
Trial summary
Hyperparameters:
activation: tanh
dropout: 0
num_layers: 7
units: 150
learning_rate: 0.0001
Score: 7.28537343093194e-05
Trial summary
Hyperparameters:
activation: selu
dropout: 0
num_layers: 2
units: 750
learning_rate: 1e-05
Score: 7.925083627924323e-05
Trial summary
Hyperparameters:
activation: tanh
dropout: 0
num_layers: 4
units: 350
learning_rate: 0.001
Score: 8.285707735922188e-05
Trial summary
Hyperparameters:
activation: tanh
dropout: 0
num_layers: 9
units: 800
learning_rate: 0.0001
Score: 8.63887180457823e

According to the Random Search method, the best hyperparameters are:
*   ReLU activation
*   9 layers
*   500 neurons
*   Dropout rate of 0
*   Learning rate of 1e-05







